In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError

/Users/int1770/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
# Load the model
MODEL_PATH = "./models/ui_evaluation_model.h5"
model = load_model(MODEL_PATH, custom_objects={'mse': MeanSquaredError()})

In [5]:
# Load the data
PROCESSED_PATH = "./preprocessing/"
images = np.load(f"{PROCESSED_PATH}images.npy")
labels = np.load(f"{PROCESSED_PATH}labels.npy")

In [6]:
# Split into training and validation sets
val_size = int(len(images) * 0.2)
val_indices = np.random.choice(len(images), val_size, replace=False)
train_indices = np.array([i for i in range(len(images)) if i not in val_indices])

val_images = images[val_indices]
val_labels = labels[val_indices]
train_images = images[train_indices]
train_labels = labels[train_indices]

In [5]:
# Print model architecture
print("--- Model Architecture ---")
model.summary()

--- Model Architecture ---


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    23,888,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,907,523 (91.20 MB)

 Trainable params: 23,907,521 (91.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [2]:
# Evaluate on training and validation data
print("\n--- Model Evaluation ---")
train_evaluation = model.evaluate(train_images, train_labels, verbose=0)
val_evaluation = model.evaluate(val_images, val_labels, verbose=0)

print(f"Training Loss (MSE): {train_evaluation[0]:.4f}")
print(f"Training Mean Absolute Error: {train_evaluation[1]:.4f}")
print(f"Validation Loss (MSE): {val_evaluation[0]:.4f}")
print(f"Validation Mean Absolute Error: {val_evaluation[1]:.4f}")


--- Model Evaluation ---


NameError: name 'model' is not defined

In [7]:
# Calculate R-squared for validation data
val_predictions = model.predict(val_images)
val_mean = np.mean(val_labels)
ss_total = np.sum((val_labels - val_mean) ** 2)
ss_residual = np.sum((val_labels - val_predictions.flatten()) ** 2)
r_squared = 1 - (ss_residual / ss_total)
print(f"R-squared: {r_squared:.4f}")

: 

In [ ]:
# Plot predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(val_labels, val_predictions, alpha=0.5)
plt.plot([min(val_labels), max(val_labels)], [min(val_labels), max(val_labels)], 'r--')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.title('Prediction vs Actual')
plt.savefig('./models/prediction_vs_actual.png')

In [ ]:
# Plot error distribution
errors = val_predictions.flatten() - val_labels
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=30)
plt.xlabel('Prediction Error')
plt.ylabel('Count')
plt.title('Error Distribution')
plt.savefig('./models/error_distribution.png')


In [ ]:
# Save metrics to file
with open("./models/model_analysis.txt", "w") as f:
    f.write("--- Model Evaluation ---\n")
    f.write(f"Training Loss (MSE): {train_evaluation[0]:.4f}\n")
    f.write(f"Training Mean Absolute Error: {train_evaluation[1]:.4f}\n")
    f.write(f"Validation Loss (MSE): {val_evaluation[0]:.4f}\n")
    f.write(f"Validation Mean Absolute Error: {val_evaluation[1]:.4f}\n")
    f.write(f"R-squared: {r_squared:.4f}\n")

print("\nAnalysis complete. Results saved to './models/model_analysis.txt'")
print("Plots saved to './models/prediction_vs_actual.png' and './models/error_distribution.png'")